In [85]:
import numpy as np
import matplotlib.pyplot as plt 
from arboles import *


In [32]:
class AST:
    def __init__(self, allowed_functions, allowed_terminals, prob_terminal=0.3):
        
        # las funciones (nodos en nuestro caso) que nuestro programa puede tener
        self.functions = allowed_functions
        # los terminales admitidos en nuestro programa. Numeros por ejemplo
        self.terminals = allowed_terminals
        # para no tener un arbol infinitamente profundo, existe una posibilidad
        # de que, a pesar de que ahora toque hacer otro sub arbol, que se ignore
        # eso y se ponga un terminal en su lugar.
        self.prob = prob_terminal

    # esta funcion ya la hemos visto, nos permite llamar al AST como si fuera
    # una funcion. max_depth es la produndidad que queremos tenga el arbol
    def __call__(self, max_depth=10):
        
        # aqui tenemos una funcion auxiliar. Nos permitira hacer esto recursivo
        def create_rec_tree(depth):
            # si `depth` es mayor a 0, nos toca crear un sub-arbol
            if depth > 0:
                # elegimos una funcion aleatoriamente
                node_cls = np.random.choice(self.functions)
                # aqui iremos dejando los argumentos que necesita la funcion
                arguments = []
                # para cada argumento que la funcion necesite...
                for _ in range(node_cls.num_args):
                    # existe un `prob` probabilidad de que no sigamos creando
                    # sub-arboles y lleguemos y cortemos aqui para hacer
                    # un nodo terminal
                    if np.random.random() < self.prob:
                        arguments.append(create_rec_tree(0))
                    else:
                        # la otra es seguir creando sub-arboles recursivamente
                        arguments.append(create_rec_tree(depth - 1))
                
                # `arguments` es una lista y los nodos necesitan argumentos
                # asi que hacemos "unpacking" de la lista
                return node_cls(*arguments)
            else:
                # si `depth` es 0 entonces creamos un nodo terminal con
                # alguno de los terminales permitidos que definimos inicialmente
                return TerminalNode(np.random.choice(self.terminals))

        # llamamos a la funcion auxiliar para crear un arbol de profundidad `max_depth`
        return create_rec_tree(max_depth)

In [33]:
a = AST([AddNode,MultNode,SubNode],[1,2,3])

In [34]:
from genetic_algorithm_mod import *

In [35]:
# Función para crear los genes de los individuos del ejercicio 1. Recibe 2 inputs:
# - gene_type: Corresponde al tipo de gen (En este caso es un bit por lo tanto es 'binary' )
# - fact_range: conjunto de valores posibles para un gen (En este caso los valores posibles son 0 o 1)
# Retorna un gen del tipo especificado y dentro del rango especificado.
def gene_factory_ex1(AST, indv_chars):
    allowed_functions = indv_chars['allowed_functions']
    allowed_terminals = indv_chars['allowed_terminals']
    prob_terminal = indv_chars['prob_terminal']
    factory = AST(allowed_functions, allowed_terminals, prob_terminal)    
    return factory

# Función para crear los individuos. Recibe 2 inputs:
# - Función creadora de genes: descrita arriba.
# - Diccionario caracterizando al individuo: diccionario que para cada gen del individuo contiene la información del
#   tipo de gen y del conjunto de valores posibles para ese gen
# Retorna un individuo adecuado para el problema.
def indv_factory(gene_factory_ex1, indv_chars): 
    factory = gene_factory_ex1(AST, indv_chars)
    max_depth = indv_chars['max_depth']
    new_indv = factory(max_depth)   
    return new_indv.copy()  # retornamos el individuo creado


# Función de fitness para el ejercicio 1. Recibe a un individuo y un diccionario con la secuencia y un ponderador.
# Retorna la multiplicación del ponderador y el valor absoluto de la diferencia entre el número entero que representa
# la secuencia entregada y el número entero que representa la secuencia del individuo.
def fitness_ex1(indv, objective):
    val = indv.eval()    
    fitness = objective['ponderador'] * abs(objective['secuencia'] - val)
    return -fitness

In [39]:
# Prueba ejercicio 1
# Seteamos los parametros a ser ocupados por el algoritmo para obtener los gráficos pedidos
secuencia_bits = 65346
ponderador = 1/50000 
fit_params = {'secuencia': secuencia_bits, 'ponderador': ponderador}
pop_sz_0 = 10
fit_fn = fitness_ex1
cr_genes = gene_factory_ex1
cr_indv = indv_factory
mut_rate = 0.1
term_cond = {'type': 'iterations', 'fitness_th': 0, 'iters': 100}
selection_type = 'tournament'
slots = 4
elitism_rate = 0.0
random_state = 42
indv_chars = {}
indv_chars['allowed_functions']=[AddNode,SubNode,MultNode,MaxNode]
indv_chars['allowed_terminals']= [25,7,8,100,4,2]
indv_chars['prob_terminal'] =0.3
indv_chars['max_depth']=10
GA = GENALG(pop_sz_0, fit_fn, cr_genes, cr_indv, indv_chars, term_cond, mut_rate, elitism_rate)
generations, goal_cross, overall_max_fitness, overall_fittest_indv = GA.apply(fit_params, selection_type,random_state, slots)
print(overall_max_fitness)
print(overall_fittest_indv)
# Obtener visualización de Evolución de fitness por generación sin elitismo
x = list(generations.keys())
vals = list(generations.values())
max_fit = [item['max_fitness'] for item in vals]
mean_fit = [item['mean_fitness'] for item in vals]
min_fit = [item['min_fitness'] for item in vals]
fig = plt.figure(figsize=(15, 15))
plt.plot(x, max_fit, marker='o', markerfacecolor='green', markersize=5, color='olive', linewidth=2, label='max_fit')
plt.plot(x, mean_fit, marker='', color='blue', linewidth=2, label='mean_fit')
plt.plot(x, min_fit, marker='', color='red', linewidth=2, label='min_fit')
plt.title("Evolución de fitness por generación", fontsize=16, fontweight='bold')
plt.suptitle("Secuencia de bits", fontsize=20)
plt.xlabel("Generación", fontsize=15)
plt.ylabel("Fitness", fontsize=15)
plt.legend(fontsize=15)
plt.savefig('Images\Fitness_por_Generacion_EX1.png')
plt.close()
print('gráfico 1 listo')

C:\Users\pclev\Documents\DATA\GIT\Tarea_3\arboles.py:114: RuntimeWarning: overflow encountered in long_scalars
  return x * y


-0.0004
(2 - (((2 - 2) + (5 * ((((2 - ((5 * 3) + (5 * 5))) + (((5 - 5) - (2 - 5)) - 2)) - (5 + (((3 + 2) + 5) - ((3 + 3) * 2)))) - (((5 - ((2 - 2) + (5 - 3))) * 3) - 5)))) * (((5 * (5 * (3 * (3 - 5)))) + (3 * ((((3 + 5) + 5) - ((3 + 5) * 2)) + (((5 + (5 - 3)) + ((5 * 3) + 2)) - ((5 - 5) - (5 + (2 - 5))))))) - (2 + ((((((5 * 2) - (3 - 2)) - (5 + 5)) * ((2 * 5) + ((5 * 5) + (2 - ((5 - 5) - (2 - 5)))))) + ((((3 * 2) + (5 + 5)) * (3 - (5 * 2))) * 3)) - (((((5 - 3) + 5) + (2 + (3 - 5))) - (((5 - 3) * (5 - 5)) * 5)) + 3))))))
gráfico 1 listo


In [ ]:
np.random.choice([1,2,3])

In [38]:
print(overall_max_fitness)
print(overall_fittest_indv)

-0.0004
(2 - (((2 - 2) + (5 * ((((2 - ((5 * 3) + (5 * 5))) + (((5 - 5) - (2 - 5)) - 2)) - (5 + (((3 + 2) + 5) - ((3 + 3) * 2)))) - (((5 - ((2 - 2) + (5 - 3))) * 3) - 5)))) * (((5 * (5 * (3 * (3 - 5)))) + (3 * ((((3 + 5) + 5) - ((3 + 5) * 2)) + (((5 + (5 - 3)) + ((5 * 3) + 2)) - ((5 - 5) - (5 + (2 - 5))))))) - (2 + ((((((5 * 2) - (3 - 2)) - (5 + 5)) * ((2 * 5) + ((5 * 5) + (2 - ((5 - 5) - (2 - 5)))))) + ((((3 * 2) + (5 + 5)) * (3 - (5 * 2))) * 3)) - (((((5 - 3) + 5) + (2 + (3 - 5))) - (((5 - 3) * (5 - 5)) * 5)) + 3))))))


In [41]:
np.mean([1,2,3])

2.0

In [43]:
type(2.3) in [int,float]

True

In [44]:
type({1:1})

dict

In [45]:
type((1))

int

In [46]:
type((1,2))

tuple

In [50]:
a=x

In [51]:
x=3

In [52]:
a

3

In [53]:
str(a)

'3'

In [54]:
a=z

NameError: name 'z' is not defined

In [55]:
str('a')

'a'

In [57]:
b=np.array([1,2,3])

In [58]:
type(b)

numpy.ndarray

In [64]:
type(b) is np.ndarray

True

In [66]:
for item in b:
    print(item)

1
2
3


In [67]:
ax = {'x':3,'y':4}

In [68]:
keys= list(ax.keys())

In [69]:
ans=True
for item in keys:
    ans*=type(ax[item])in [int,float]
    

In [70]:
ans

1

In [71]:
if ans:
    print('yes')

yes


In [72]:
bool(1)

True

In [ ]:
bo

In [81]:
bool(np.product([1,1,0]))

False

In [79]:
np.product([1,1,0])

0